In [1]:
import pandas as pd
import numpy as np

train_df1 = pd.read_csv('Desktop/比赛数据特征/train/train_df1.csv')
train_df2 = pd.read_csv('Desktop/比赛数据特征/train/train_df2.csv')
train_df3 = pd.read_csv('Desktop/比赛数据特征/train/train_df3.csv')
train_df4 = pd.read_csv('Desktop/比赛数据特征/train/train_df4.csv')
train_df5 = pd.read_csv('Desktop/比赛数据特征/train/train_df5.csv')
train_df6 = pd.read_csv('Desktop/比赛数据特征/train/train_df6.csv')
train_df7 = pd.read_csv('Desktop/比赛数据特征/train/train_df7.csv')
train_df8 = pd.read_csv('Desktop/比赛数据特征/train/train_df8.csv')

test_df1 = pd.read_csv('Desktop/比赛数据特征/test/test_df1.csv')
test_df2 = pd.read_csv('Desktop/比赛数据特征/test/test_df2.csv')
test_df3 = pd.read_csv('Desktop/比赛数据特征/test/test_df3.csv')
test_df4 = pd.read_csv('Desktop/比赛数据特征/test/test_df4.csv')
test_df5 = pd.read_csv('Desktop/比赛数据特征/test/test_df5.csv')
test_df6 = pd.read_csv('Desktop/比赛数据特征/test/test_df6.csv')
test_df7 = pd.read_csv('Desktop/比赛数据特征/test/test_df7.csv')

train = pd.merge(train_df1, train_df2, on='EID', how='left')
train = pd.merge(train, train_df3, on='EID', how='left')
train = pd.merge(train, train_df4, on='EID', how='left')
train = pd.merge(train, train_df5, on='EID', how='left')
train = pd.merge(train, train_df6, on='EID', how='left')
train = pd.merge(train, train_df7, on='EID', how='left')
train = pd.merge(train, train_df8, on='EID', how='left')

test = pd.merge(test_df1, test_df2, on='EID', how='left')
test = pd.merge(test, test_df3, on='EID', how='left')
test = pd.merge(test, test_df4, on='EID', how='left')
test = pd.merge(test, test_df5, on='EID', how='left')
test = pd.merge(test, test_df6, on='EID', how='left')
test = pd.merge(test, test_df7, on='EID', how='left')

#train.shape

#test.shape

train['TARGET'] = train.TARGET.fillna(0)

#train.TARGET.value_counts()

#19882+368

#368/(368+19882)

#19882/(368+19882)

labels = train.TARGET

In [2]:
train.drop(['TARGET'],axis=1,inplace=True)

In [3]:
all_feat = pd.concat([train, test]).reset_index(drop=True)

In [4]:
all_feat.shape

(32767, 67)

In [5]:
all_feat.fillna(0,inplace=True)

In [6]:
all_feat['EID'] = all_feat['EID'].apply(lambda x: int(x[1:]))

In [7]:
#all_feat.head(2)

In [8]:
all_feat = pd.get_dummies(all_feat)

In [9]:
train_count = train.shape[0]
test_count = test.shape[0]

In [10]:
train_feat = all_feat[:train_count].reset_index(drop=True)
test_feat = all_feat[-test_count:].reset_index(drop=True)

In [11]:
train_feat.shape

(20250, 129)

In [12]:
test_feat.shape

(12517, 129)

In [13]:
#train_feat.head(2)

In [14]:
#test_feat.head(2)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score,roc_auc_score

# feature selection

In [16]:
clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
clf = clf.fit(train_feat, labels)

In [17]:
model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(train_feat)
print(train_reduced.shape)

(20250, 43)


In [18]:
test_reduced = model.transform(test_feat)
print(test_reduced.shape)

(12517, 43)


# auto-sklearn

In [19]:
from autosklearn import classification
import warnings
warnings.simplefilter('ignore', RuntimeWarning)

In [20]:
automl = classification.AutoSklearnClassifier(
    include_estimators=['random_forest','xgradient_boosting'], exclude_estimators=None,
    include_preprocessors=['no_preprocessing', ], exclude_preprocessors=None
)

In [ ]:
include_estimators=['adaboost',
                   'bernoulli_nb',
                   'decision_tree',
                   'extra_trees',
                   'gaussian_nb',
                   'gradient_boosting',
                   'k_nearest_neighbors',
                   'lda',
                   'liblinear_svc',
                   'libsvm_svc',
                   'multinomial_nb',
                   'passive_aggressive',
                   'qda',
                   'random_forest',
                   'sgd',
                   'xgradient_boosting']

In [21]:
automl.fit(train_reduced,labels)

[WARNING] [2018-10-09 16:45:52,207:EnsembleBuilder(1):f0519e2437d638a3c9e0dff917156f77] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-09 16:45:52,213:EnsembleBuilder(1):f0519e2437d638a3c9e0dff917156f77] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-09 16:45:54,216:EnsembleBuilder(1):f0519e2437d638a3c9e0dff917156f77] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-09 16:45:57,948:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-09 16:45:57,948:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


AutoSklearnClassifier(delete_output_folder_after_terminate=True,
           delete_tmp_folder_after_terminate=True,
           disable_evaluator_output=False, ensemble_nbest=50,
           ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=['random_forest', 'xgradient_boosting'],
           include_preprocessors=None,
           initial_configurations_via_metalearning=25,
           ml_memory_limit=3072, output_folder=None,
           per_run_time_limit=360, resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

## training set auc score

In [22]:
train_pred = automl.predict(train_reduced).astype(int)

In [23]:
print(np.unique(train_pred))

[0 1]


In [24]:
prob_train_pred = automl.predict_proba(train_reduced)

prob_train_pred = [p[1] for p in prob_train_pred]

print(roc_auc_score(labels,prob_train_pred))

0.9860339043836898


## training set accuracy score

In [31]:
from sklearn.metrics import accuracy_score

y_hat = automl.predict(train_reduced)
print("Accuracy score", accuracy_score(labels, y_hat))

Accuracy score 0.9910617283950617


## test

In [32]:
test_pred = automl.predict(test_reduced).astype(int)

In [33]:
print(np.unique(test_pred))

[0 1]


In [34]:
prob_test_pred = automl.predict_proba(test_reduced)
prob_test_pred = [p[1] for p in prob_test_pred]

In [35]:
pd.DataFrame({'EID': test.EID, 'FORTARGET': test_pred, 'PROB': prob_test_pred}).to_csv(
    'Desktop/automl_test.csv', index=False)

# model persistence

In [36]:
from sklearn.externals import joblib

In [37]:
joblib.dump(automl,'Desktop/automl_xinyong.joblib')

['Desktop/automl_xinyong.joblib']

In [38]:
autoclf = joblib.load('Desktop/automl_xinyong.joblib')

In [39]:
from sklearn.metrics import accuracy_score

y_hat = autoclf.predict(train_reduced)
print("Accuracy score", accuracy_score(labels, y_hat))

Accuracy score 0.9910617283950617
